rpc permissions?


In [ ]:

// Always return the permission set so it can't be manipulated by another command.
function getPermissions() {
    return {
        'google contacts.ipynb[google contact settings]': ['Inner Circle', 'Public'],
        'import notebook.ipynb[load interpreter]' : ['Public'],
        'rpc.ipynb[permissions]' : ['Public'],
        'rpc.ipynb[parameter names]' : ['Public'],
        'gulp.ipynb[search notebooks]' : ['Public'],
        'data collection.ipynb[search all]' : ['Public'],
        'data collection.ipynb[get joke]' : ['Public'],
    };
}
module.exports = getPermissions;


filter command permission?


In [ ]:
var importer = require('../Core');
var {
    getPermissions,
    getSettings,
    getDaysEvents,
    getOauthClient,
} = importer.import([
    'rpc permissions',
    'days events',
    'google contact settings',
    'import google calendar api'
]);

var options = {
    calendarId: 'Commands'
};

// TODO: move this logic out to a higher level coordinator?
function filterCommand(command, date, id, user) {
    const props = {};
    return getOauthClient(options)
        .then(() => getSettings(user))
        .then(settings => {
            // filter permissions
            Object.assign(props, settings);
            try {
                console.log(settings.circles)
                // compare id with permissions
                Object.assign(props, {
                    result: importer.interpret(command)
                });
                Object.assign(props, {
                    allowed: settings.circles
                        .filter(c => {
                            const matching = Object.keys(getPermissions())
                                .filter(k => getPermissions()[k].indexOf(c) > -1 || getPermissions()[k].indexOf('Public') > -1);
                            return matching.filter(k => importer.interpret(k).id === props.result.id).length > 0;
                        }).length > 0
                });
                // if looking for settings use the user name provided to this function for lookup
                // TODO: accept parameters from message context
                if(props.result.id === importer.interpret('google contacts.ipynb[google contact settings]').id) {
                    props.result.runInNewContext = () => settings;
                }
            } catch (e) {
                if((e + '').indexOf('Nothing found') > -1) {
                    Object.assign(props, {result: null});
                } else {
                    throw e;
                }
            }
        })
        .then(() => getDaysEvents(new Date(date), options))
        .then(events => {
            // filter already sent
            const already = events
              .filter(e => e.event.summary.indexOf('Result:') > -1 && e.event.summary.indexOf(id) > -1).length > 0;
            Object.assign(props, {already: already})
        })
        .catch(e => console.log(e))
        .then(() => props)
}
module.exports = filterCommand;


store rpc result?


In [ ]:
var importer = require('../Core');
var {
    updateEvent,
    ISODateString,
    getOauthClient,
} = importer.import([
    'convert date iso',
    'update create merge event',
    'import google calendar api'
]);

var options = {
    calendarId: 'Commands'
};

function storeResult(friend) {
    var result;
    if(typeof friend === 'undefined' || friend === null
       || friend.already !== false) {
        // skip commands that have already been run
        throw new Error('Nothing to do!');
    }
    
    var isEmail = false;
    if(typeof friend.subject !== 'undefined') {
        isEmail = true;
    }
    
    if (friend.result === null) {
        result = Promise.resolve('Command not found, please specify a command ' + (isEmail ? 'in the subject line' : '') + ' like "megamind settings" without quotes.')
    } else if(!friend.allowed) {
        result = Promise.resolve('Would have run "' + friend.result.id + '" but you don\'t have permission.');
    } else {
        const commandResult = friend.result.runInNewContext();
        result = Promise.resolve([])
            .then(() => typeof commandResult === 'function' ? commandResult() : commandResult)
            .catch(e => Object.getOwnPropertyNames(e).reduce((alt, key) => {
                alt[key] = e[key];
                return alt;
            }, {}))
    }

    var response;
    return getOauthClient(options)
        .then(() => result)
        .then(r => {
            response = r;
            // store as an event for tracking
            const config = {
                start: {
                    dateTime: ISODateString(new Date(friend.date.getTime()))
                },
                end: {
                    dateTime: ISODateString(new Date(friend.date.getTime() + 60 * 30 * 1000))
                },
                summary: 'Result: ' + (isEmail ? 'Email: ' : '') + friend.id,
                description: JSON.stringify(response, null, 4)
            }
            return updateEvent(config, options);
        })
        .then(() => response)
        .catch(e => console.log(e))
}
module.exports = storeResult;



get parameter names?

function parameters?


In [ ]:
var importer = require('../Core');

var COMMENTS = /((\/\/.*$)|(\/\*[\s\S]*?\*\/))/mg;
var DEFAULT_PARAMS = /=[^,]+/mg;
var FAT_ARROWS = /=>[\s\S]*$|{[\s\S]*$/mg;

function getParameterNames(search) {
    var result;
    if(typeof search === 'function') {
        result = search;
    } else {
        result = importer.import(search);
    }

    if(typeof result === 'object' && result !== null
        && typeof result[Object.keys(result)[0]] === 'function') {
        result = result[Object.keys(result)[0]];
    }
    var fnName = result.name;
    
    var code = result.toString()
        .replace(COMMENTS, '')
        .replace(FAT_ARROWS, '')
        .replace(DEFAULT_PARAMS, '');

    var result = code.slice(code.indexOf('(') + 1, code.indexOf(')'))
        .match(/([^\s,]+)/g);

    return result === null
        ? [fnName]
        : [fnName, ...result];
}

module.exports = getParameterNames;


firebase rpc wrapper?


In [3]:
var cors = require('cors')({origin: true});
var importer = require('../Core');
var functions = require('firebase-functions'),
    admin = require('firebase-admin');

try {
    admin.initializeApp(functions.config().firebase);
} catch (e) {
    console.log(e);
}

var {
    getPermissions,
    getParameterNames
} = importer.import([
    'rpc permissions',
    'function parameters'
]);
var rpcFunctions = Object.keys(getPermissions())
    .filter(k => getPermissions()[k].indexOf('Public') > -1);

console.log(rpcFunctions);

if(typeof module.exports === 'undefined') {
    module.exports = {};
}

function addRoute(search) {
    const parameters = getParameterNames(search);
    console.log(parameters);
    module.exports[parameters[0]] = functions.https.onRequest((req, res) => {
        cors(req, res, () => {
            // TODO: get parameters from query
            const result = importer.import(search);
            console.log('Starting ' + parameters.join(' '))
            return Promise.resolve(result.apply(importer, parameters.slice(1).map(k => req.query[k] || req.body[k])))
                .then(r => res.send(JSON.stringify(r, null, 4)))
                // TODO: object assign error?
                .catch(e => res.send(e + ''));
        });
    });
}

for(var i = 0; i < rpcFunctions.length; i++) {
    addRoute(rpcFunctions[i]);
}


ReferenceError: admin is not defined
    at evalmachine.<anonymous>:6:5
    at ContextifyScript.Script.runInThisContext (vm.js:50:33)
    at Object.runInThisContext (vm.js:139:38)
    at run ([eval]:775:15)
    at onRunRequest ([eval]:620:18)
    at onMessage ([eval]:588:13)
    at emitTwo (events.js:125:13)
    at process.emit (events.js:213:7)
    at emit (internal/child_process.js:774:12)
    at _combinedTickCallback (internal/process/next_tick.js:141:11)
[ 'google contacts.ipynb[google contact settings]',
  'import notebook.ipynb[load interpreter]',
  'rpc.ipynb[permissions]' ]
[ 'name' ]
[ 'query' ]
[]
